In [1]:
### Libraries
import numpy as np
import ast
import urllib
import matplotlib.pyplot as plt
import random
from collections import defaultdict
import scipy.optimize
from sklearn import linear_model
import math
from sklearn.ensemble import AdaBoostClassifier
import gzip
import string

In [2]:
def parseDataFromFile(fname):
    """ Read and convert the input to a list of dicts"""
    for l in open(fname, encoding="utf-8"):
        yield ast.literal_eval(l)
        
def readGz(path):
    for l in gzip.open(path, 'rt', encoding="utf-8"):
        yield eval(l)

In [3]:
fname = "C:\\Users\\ramasarma\\Documents\\UCSD\\Fall 2020\\CSE 258\\Assignment1\\assignment1\\train_Category.json"
data = list(parseDataFromFile(fname))
print(len(data))

175000


In [4]:
print(data[0])
# Considering only the first 10000 samples
dataset = data[:10000]

{'userID': 'u74382925', 'genre': 'Adventure', 'early_access': False, 'reviewID': 'r75487422', 'hours': 4.1, 'text': 'Short Review:\nA good starting chapter for this series, despite the main character being annoying (for now) and a short length. The story is good and actually gets more interesting. Worth the try.\nLong Review:\nBlackwell Legacy is the first on the series of (supposedly) 5 games that talks about the main protagonist, Rosangela Blackwell, as being a so called Medium, and in this first chapter we get to know how her story will start and how she will meet her adventure companion Joey...and really, that\'s really all for for now and that\'s not a bad thing, because in a way this game wants to show how hard her new job is, and that she cannot escape her destiny as a medium.\nMy biggest complain for this chapter, except the short length, it\'s the main protagonist being a "bit" too annoying to be likeable, and most of her dialogues will always be about complaining or just be a

In [5]:
corpus = []
punctuation = set(string.punctuation)
for d in dataset:
    r = ''.join([c for c in d['text'].lower() if c not in punctuation])
    corpus.append(r)
#print(corpus[:2])

Stats related to the dataset

In [6]:
# Reading through the text after removing punctuation
wordCount = defaultdict(int)
for document in corpus:
    for word in document.split():
        wordCount[word] += 1
print("Number of unique unigrams are {}".format(len(wordCount.keys())))

Number of unique unigrams are 29692


### Question 1 - Number of unique bigrams

In [7]:
bigram_map = defaultdict(int)
for document in corpus:
    words = document.split()
    for i in range(len(words) - 1):
        first_word = words[i]
        second_word = words[i+1]
        key = first_word + ' ' + second_word
        bigram_map[key] += 1

wordCount = [(bigram_map[key], key) for key in bigram_map]
wordCount.sort()
wordCount.reverse()

print("Number of unique bigrams amongst the reviews are {}".format(len(bigram_map.keys())))

print(" The 5 top bigrams are mentioned as follows - ")
print(wordCount[:5])

Number of unique bigrams amongst the reviews are 256618
 The 5 top bigrams are mentioned as follows - 
[(4441, 'this game'), (4249, 'the game'), (3359, 'of the'), (2020, 'if you'), (2017, 'in the')]


### Question 2 - Here, we are using the 1000 Most common Bigrams as features.

In [8]:
topNGrams = wordCount[:1000]
bigrams = [val[1] for val in topNGrams]
bigramId = dict(zip(bigrams, range(len(bigrams))))
bigramSet = set(bigrams)

def feature(datum):
    feat = ([0]*len(bigrams))
    r = ''.join([c for c in datum['text'].lower() if not c in punctuation])
    words = r.split()
    for i in range(len(words) - 1):
        bigram = words[i] + ' ' + words[i + 1]
        if bigram in bigrams:
            feat[bigramId[bigram]] += 1
    feat.append(1)
    return feat

X = [feature(d) for d in dataset]
y = [math.log2(d['hours'] + 1) for d in dataset]

clf = linear_model.Ridge(1.0, fit_intercept=False) # MSE + 1.0 l2
clf.fit(X, y)
theta = clf.coef_
predictions = clf.predict(X)

In [9]:
def MSE(actual, predictions):
    differences = [(x-y)**2 for (x, y) in zip(actual, predictions)]
    return sum(differences)/len(differences)

# Computing the mean squared error
mse_train = MSE(y, predictions)
print("Training set MSE = {}".format(mse_train))

Training set MSE = 4.393787247200031


### Question 3 - Here, we are using the 1000 most common unigrams and bigrams as features

In [10]:
def feature(datum, K):
    """ args : datum - data sample
        K : Top K Grams to be filtered """
    r = ''.join([c for c in datum['text'].lower() if c not in punctuation])
    words = r.split()
    if(len(words) == 0):
        feat = [0] * K
        feat.append(1)
        return feat
    bigram_map = defaultdict(int)
    unigram_map = defaultdict(int)
    
    # Bigram map is used to store bigram frequencies
    # Unigram map is used to store unigram frequencies
    for i in range(len(words) - 1):
        bigram_key = words[i] + ' ' + words[i+1]
        bigram_map[bigram_key] += 1
        unigram_map[words[i]] += 1
    #last_word = words[-1]
    unigram_map[words[-1]] += 1
    
    # Compute the same for unigram and bigrams
    wordCount_unigram = [(unigram_map[key], key) for key in unigram_map]
    wordCount_bigram = [(bigram_map[key], key) for key in bigram_map]
    
    # Compute the wordCount for unigram and bigram
    wordCount_unigram.sort()
    wordCount_bigram.sort()
    wordCount_unigram.reverse()
    wordCount_bigram.reverse()
    
    # List of keys for the unigram and bigram
    unigramKeys = [x[1] for x in wordCount_unigram]
    bigramKeys = [x[1] for x in wordCount_bigram]
    unigramId = dict(zip(unigramKeys, range(len(unigramKeys))))
    bigramId = dict(zip(bigramKeys, range(len(bigramKeys))))
    
    # Feature of a combination of unigram and bigrams
    topNGrams = []
    
    # Problem similar to merging two sorted lists with a condition on the max number of lists
    i, i_u, i_b = 0, 0, 0
    while (i < K and (i_u < len(wordCount_unigram) and i_b < len(wordCount_bigram))):
        if(wordCount_unigram[i_u][0] > wordCount_bigram[i_b][0]):
            topNGrams.append(wordCount_unigram[i_u][1])
            i_u += 1
        else:
            topNGrams.append(wordCount_bigram[i_b][1])
            i_b += 1
        i += 1

    while (i < K and (i_u < len(wordCount_unigram))):
        topNGrams.append(wordCount_unigram[i_u][1])
        i_u += 1
        i += 1

    while (i < K and (i_b < len(wordCount_bigram))):
        topNGrams.append(wordCount_bigram[i_b][1])
        i_b += 1
        i += 1
        
    feat = ([0] * K)
    nGrams = [val for val in topNGrams]
    nGramId = dict(zip(nGrams, range(len(nGrams))))
    nGramSet = set(nGramId)
    for nGram in topNGrams:
        # Logic to check whether it's a bigram or unigram?
        if(len(nGram.split()) > 1):
            feat[nGramId[nGram]] += 1
        else:
            assert(len(nGram.split()) == 1)
            feat[nGramId[nGram]] += 1
    feat.append(1)
    return feat

# Compute the feature matrix "X" for the given dataset
X = [feature(d, 1000) for d in dataset]
# Process the dataset for the matrix "Y"
y = [math.log2(d['hours'] + 1) for d in dataset]
# Compute the Linear Regression model here
clf = linear_model.Ridge(1.0, fit_intercept=True) # MSE + 1.0 l2
clf.fit(X, y)
theta = clf.coef_
predictions = clf.predict(X)

mse_train = MSE(y, predictions)
print("Training set MSE = {}".format(mse_train))

Training set MSE = 4.9110892467357665


### Question 4 - Inverse Document Frequency

In [29]:
def computeIDF(corpus, searches):
    idfs = []
    numDocs = len(corpus)
    for searchWord in searches:
        count = 0
        for document in corpus:
            words = searchWord.split()
            if(searchWord in words):
                count += 1
        idfs.append(math.log10(numDocs/(1 + count)))
    return idfs

def computeTF(corpus, searches):
    (rows, cols) = (len(corpus), len(searches))
    tfs = [[0 for i in range(cols)] for j in range(rows)]
    for searchWord in searches:
        for document in corpus:
            words = document.split()
            tf, numWords = 0, len(words)
            word_idx = searches.index(searchWord)
            doc_idx = corpus.index(document)
            if(searchWord in words):
                tfs[doc_idx][word_idx] += 1/numWords
    
    return tfs

# Go through the entire list of searches to compute the IDF
searches = ["destiny", "annoying", "likeable", "chapter", "interesting"]
idf = computeIDF(corpus, searches)
print("First five IDFs are {}".format(idf[:5]))
tf = computeTF(corpus, searches)
print("First five TFs are {}".format(tf[:5]))

First five IDFs are [-4.342727686267685e-05, -4.342727686267685e-05, -4.342727686267685e-05, -4.342727686267685e-05, -4.342727686267685e-05]
First five TFs are [[0.004694835680751174, 0.004694835680751174, 0.004694835680751174, 0.004694835680751174, 0.004694835680751174], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0], [0, 0, 0, 0, 0.007042253521126761], [0, 0, 0, 0, 0]]


### TF IDF scores for the reviewID r75487422

In [30]:
# review provides the entity in corpus
review = corpus[0]
tf, tf_idf = [0]*len(searches), [0] * len(searches)
for i in range(len(searches)):
    words = review.split()
    numWords = len(words)
    # Number of words = length of words in a document
    count = 0
    if (searches[i] in words):
        count += 1
    tf[i] = count / numWords
    val = 1 if (tf[i] > 0) else 0
    tf_idf[i] = tf[i] * idf[i]
    print("Word = {}, TF score = {}, IDF score = {}, TFIDF score = {}".format(searches[i], tf[i], idf[i], tf_idf[i]))

Word = destiny, TF score = 0.004694835680751174, IDF score = -4.342727686267685e-05, TFIDF score = -2.0388392893275517e-07
Word = annoying, TF score = 0.004694835680751174, IDF score = -4.342727686267685e-05, TFIDF score = -2.0388392893275517e-07
Word = likeable, TF score = 0.004694835680751174, IDF score = -4.342727686267685e-05, TFIDF score = -2.0388392893275517e-07
Word = chapter, TF score = 0.004694835680751174, IDF score = -4.342727686267685e-05, TFIDF score = -2.0388392893275517e-07
Word = interesting, TF score = 0.004694835680751174, IDF score = -4.342727686267685e-05, TFIDF score = -2.0388392893275517e-07


### Adapt the unigram model to use the TF-IDF scores

### Question 5

In [31]:
def computeTopNUnigrams(corpus, N):
    topNUnigrams = []
    wordCount = defaultdict(int)
    for document in corpus:
        for word in document.split():
            wordCount[word] += 1
    
    mostCommon = [(wordCount[x], x) for x in wordCount]
    # Sort and reverse the code
    mostCommon.sort()
    mostCommon.reverse()
    for i in range(N):
        topNUnigrams.append(mostCommon[i][1])
    return topNUnigrams

def computeTFIDF(tf, idf):
    len_tf = len(tf)
    tf_idfs = []
    for i in range(len_tf):
        tf_idf = []
        for j in range(len(tf[i])):
            tf_idf.append(float(tf[i][j] * idf[j]))
        tf_idfs.append(tf_idf)
    return tf_idfs

In [32]:
def feature(corpus, N):
    """ feature() is used to compute the features """
    topNUnigrams = computeTopNUnigrams(corpus, N)
    #print(top1000Unigrams[:5])
    idf = computeIDF(corpus, topNUnigrams)
    tf = computeTF(corpus, topNUnigrams)
    tf_idf = computeTFIDF(tf, idf)
    for i in range(len(tf_idf)):
        # Offset addition
        tf_idf[i].append(1)
    return tf_idf

X_train = feature(corpus, 1000)
Y_train = [math.log2(d['hours'] + 1) for d in dataset]
print("Computed features for the training set")
print(len(X_train), len(X_train[0]))
clf = linear_model.Ridge(1.0, fit_intercept=False) # MSE + 1.0 l2
clf.fit(X_train, Y_train)
theta = clf.coef_
predictions = clf.predict(X_train)
mse_train = MSE(Y_train, predictions)
print("Training set MSE = {}".format(mse_train))

Computed features for the training set
10000 1001
Training set MSE = 5.242479008006355


### Question 6

In [33]:
def norm(A):
    val = 0
    for ele in A:
        val += ele**2
    return val

# Here, we have the cosine similarity being defined
def cosine_similarity(A, B):
    numerator = np.dot(A, B)
    denominator = math.sqrt(norm(A) * norm(B))
    return numerator/denominator

# For every datapoint in the given dataset
r_idx = -1
N = 1000 # Number of unigrams to process
X = feature(corpus, 1000)
for d in dataset:
    if(d['reviewID'] == "r19740876"):
        r_idx = dataset.index(d)
        break
print("The index of review r75487422 is {}".format(r_idx))
max_sim = -1
mostSimilarReviewId = -1
max_idx = -1
for i in range(len(dataset)):
    if(i != r_idx):
        cos_sim = cosine_similarity(X[r_idx], X[i])
        if(cos_sim > max_sim):
            mostSimilarReviewId = dataset[i]['reviewID']
            max_sim = cos_sim
            max_idx = i

print("Most similar reviewID to r75487422 is {} and is present at index {}".format(mostSimilarReviewId, max_idx))

The index of review r75487422 is 8328
Most similar reviewID to r75487422 is r26288269 and is present at index 420


In [34]:
print(dataset[8328])

{'userID': 'u46449878', 'genre': 'RPG', 'early_access': False, 'reviewID': 'r19740876', 'hours': 0.6, 'text': 'NOTE: This game is Free To Play, meaning this review and many others do not count. I cannot stress enough that you play it for yourself with an open mind to get an undilluted opinion and ignore this review and any others.\nSo... I know what Depression is. I\'ve been diagnosed with it for the last 15 years alongside High Anxiety for all that time.\nYou want to know what Depression is like? Let me tell you instead of you playing this game.\nImagine sadness. Imagine extreme self-disappointment. Imagine self-loathing to as high of a degree as it\'ll go. It\'s not always the same, but to my knowledge... Mix them into one emotion and you get Depression.\nIt is literally the worst emotion on the planet. Mostly all suicides are caused by feelings of Depression. I have no study, no way to prove that, other than just plain logic and understanding.\nThis game does not convey the feelings

### Question 7

In [35]:
def computeTopNBigrams(corpus, N):
    topNBigrams = []
    wordCount = defaultdict(int)
    for document in corpus:
        words = document.split()
        for i in range(len(words)-1):
            key = word[i] + ' ' + word[i+1]
            wordCount[key] += 1
    
    mostCommon = [(wordCount[x], x) for x in wordCount]
    # Sort and reverse the code
    mostCommon.sort()
    mostCommon.reverse()
    
    for i in range(N):
        topNBigrams.append(mostCommon[i][1])
    return topNBigrams

In [36]:
print(set(string.punctuation))

{'\\', ';', '$', '`', '/', '!', '%', ']', ':', '|', '-', '+', '"', '=', '{', '^', '~', '*', "'", ',', '_', ')', '#', '>', '(', '&', '@', '}', '[', '.', '?', '<'}


### Comparison between unigram and bigram models for different variants of regularization parameters

Unigram + Removed Punctuation + TD IDF Scores = 0, 0, 0

Unigram + Removed Punctuation + Word Counts = 0, 0, 1

Unigram + Preserve Punctuation + TD IDF Scores = 0, 1, 0

Unigram + Preserve Punctuation + Word Counts = 0, 1, 1

Bigram  + Removed Punctuation + TD IDF Scores = 1, 0, 0

Bigram + Removed Punctuation + Word Counts = 1, 0, 1

Bigram + Preserve Punctuation + TD IDF Scores = 1, 1, 0

Bigram + Preserve Punctuation + Word Counts = 1, 1, 1

In [45]:
import re

def computeCount(wordCorpus, gram_type, N):
    """ args - wordCorpus - 2D list (each row - list of words in one review)
                gram_type - Unigram (or) Bigram
                N - Number of grams to search for """
    features = []
    for words in wordCorpus:
        length = len(words) if (gram_type == "unigram") else len(words) - 1
        wordCount = defaultdict(float)
        for i in range(length):
            if(gram_type == "unigram"):
                wordCount[words[i]] += 1
            else:
                wordCount[words[i] + ' ' + words[i+1]] += 1
        mostCommon = [(wordCount[x], x) for x in wordCount]
        mostCommon.sort()
        mostCommon.reverse()
        topNGrams = [] # store for every document in the corpus
        for i in range(N):
            if(i < len(mostCommon)):
                topNGrams.append(mostCommon[i][1])
        #print("Length of topNGrams = {}".format(len(topNGrams)))
        # Compute the unigramId and accordingly prepare the feature vector
        ngramId = dict(zip(topNGrams, range(len(topNGrams))))
        feature = [0] * N
        for word in words:
            if(word in topNGrams):
                feature[ngramId[word]] += 1
        feature.append(1) # For the constant \theta_0
        features.append(feature)
        
    return features

def computeTFIDF(wordCorpus, gram_type, N):
    """ args - wordCorpus - 2D list (each row - list of words in one review)
                gram_type - Unigram (or) Bigram
                N - Number of grams to search for """
    features, tfs, idfs = [], [], []
    numDocuments = defaultdict(int)
    for words in wordCorpus:
        length = len(words) if (gram_type == "unigram") else len(words) - 1
        for i in range(length):
            if(gram_type == "unigram"):
                numDocuments[words[i]] += 1
            else:
                numDocuments[words[i] + ' ' + words[i+1]] += 1
    
    for words in wordCorpus:
        # Logic to compute most frequent unigrams or bigrams
        length = len(words) if (gram_type == "unigram") else len(words) - 1
        wordCount = defaultdict(float)
        for i in range(length):
            if(gram_type == "unigram"):
                wordCount[words[i]] += 1
                numDocuments[words[i]] += 1
            else:
                wordCount[words[i] + ' ' + words[i+1]] += 1
                numDocuments[words[i] + ' ' + words[i+1]] += 1
        
        mostCommon = [(wordCount[x], x) for x in wordCount]
        mostCommon.sort()
        mostCommon.reverse()
        topNGrams = [] # store for every document in the corpus
        for i in range(N):
            if(i < len(mostCommon)):
                topNGrams.append(mostCommon[i][1])
        
        ngramId = dict(zip(topNGrams, range(len(topNGrams))))
        # Compute TF
        tf = [0] * N
        numWords = len(words)
        for word in words:
            if(word in ngramId):
                tf[ngramId[word]] += 1/numWords
        # Compute IDF
        # Number of documents that have the term present in it
        idf = [0] * N
        for i in range(N):
            if(i < len(topNGrams)):
                idf[i] = math.log10(len(wordCorpus)/(numDocuments[topNGrams[i]] + 1))
        feature = [tf[i] * idf[i] for i in range(N)]
        # For constant \theta_0
        feature.append(1)
        features.append(feature)

    return features

# Customized implementation of split function
def customSplit(characters, delimiters):
    result = []
    word = ""
    for char in characters:
        if(char in delimiters):
            if(len(word) > 0):
                result.append(word)
            if(char != ' '):
                word = str(char)
                result.append(word)
            word = ""
        else:
            word += char
    return result

def computeFeatures(dataset, gram_type, punct_type, score_type, N):
    """ dataset - Input data for which the feature needs to be extracted
        gram_type - Unigram or Bigram
        punct_type - Preserve or remove punctuations
        score_type - TF-IDF or wordCount """
    features, wordCorpus, corpus = [], [], []
    for d in dataset:
        if(punct_type == "remove"):
            review = ''.join([c for c in d['text'].lower() if c not in punctuation])
        elif(punct_type == "preserve"):
            review = ''.join([c for c in d['text'].lower()])
        corpus.append(review)
    punctList = [ch for ch in string.punctuation]
    # For every document in the list of all documents
    for document in corpus:
        delimiters = [' ']
        if(punct_type == "preserve"):
            for delimit in string.punctuation:
                delimiters.append(delimit)
            words = customSplit(document, delimiters)
        else:
            assert(punct_type == "remove")
            words = customSplit(document, delimiters)
        wordCorpus.append(words)
        
    if(score_type == "wordCount"):
        features = computeCount(wordCorpus, gram_type, N)
    else:
        assert(score_type == "tf_idf")
        features = computeTFIDF(wordCorpus, gram_type, N)
    
    return features

In [48]:
def validation_pipeline(data):
    random.seed(0)
    random.shuffle(data)
    # size = 10000
    training_set = data[:10000]
    validation_set = data[10000:20000]
    test_set = data[20000:30000]
    # Regularization parameters
    lambdas = [10 ** i for i in np.arange(-2,3,dtype=float)]
    # Different options for the loop
    gramTypes = ["unigram", "bigram"]
    punctuationTypes = ["preserve", "remove"]
    countTypes = ["tf_idf", "wordCount"]
    # Place to store MSE values for all the models
    mseVals = defaultdict(float)
    bestModels = defaultdict(float)
    # For each gramType in the list of gramTypes
    for gramType in gramTypes:
        for puncType in punctuationTypes:
            for countType in countTypes:
                # Training set is as follows
                X_train = computeFeatures(training_set, gramType, puncType, countType, 1000)
                Y_train = [math.log2(d['hours'] + 1) for d in training_set]
                # Validation set is as follows
                X_val = computeFeatures(validation_set, gramType, puncType, countType, 1000)
                Y_val = [math.log2(d['hours'] + 1) for d in validation_set]
                # Test set is as follows
                X_test = computeFeatures(test_set, gramType, puncType, countType, 1000)
                Y_test = [math.log2(d['hours'] + 1) for d in test_set]
                # Ensure that the length of the training set "equals" 1001
                assert(len(X_train[0]) == 1001)
                minMSE, minlamb = 10000, -1
                for lamb in lambdas:
                    clf = linear_model.Ridge(lamb, fit_intercept=True)
                    clf.fit(X_train, Y_train)
                    #theta = clf.coef_
                    Y_val_pred = clf.predict(X_val)
                    mse_val = MSE(Y_val, Y_val_pred)
                    key = str(gramType) + '|' + str(puncType) + '|' + str(countType) + '|' + str(lamb)
                    print("Validation MSE with key = {} is {}".format(key, mse_val))
                    mseVals[key] = mse_val
                    if(mse_val < minMSE):
                        minMSE = mse_val
                        minlamb = lamb
                k = str(gramType) + '|' + str(puncType) + '|' + str(countType) + '|' + str(minlamb)
                bestModels[k] = minMSE
                clf = linear_model.Ridge(minlamb, fit_intercept=True)
                clf.fit(X_train, Y_train)
                Y_test_pred = clf.predict(X_test)
                mse = MSE(Y_test, Y_test_pred)
                print("Test MSE at {}, {}, {} and lambda = {} is {}".\
                      format(gramType, puncType, countType, minlamb, mse))
                
validation_pipeline(data)

Validation MSE with key = unigram|preserve|tf_idf|0.01 is 5.357231999710872
Validation MSE with key = unigram|preserve|tf_idf|0.1 is 5.332903487594727
Validation MSE with key = unigram|preserve|tf_idf|1.0 is 5.321886691089576
Validation MSE with key = unigram|preserve|tf_idf|10.0 is 5.320347749905078
Validation MSE with key = unigram|preserve|tf_idf|100.0 is 5.317046585844409
Test MSE at unigram, preserve, tf_idf and lambda = 100.0 is 5.198895607621101
Validation MSE with key = unigram|preserve|wordCount|0.01 is 5.514462641035678
Validation MSE with key = unigram|preserve|wordCount|0.1 is 5.489822087879586
Validation MSE with key = unigram|preserve|wordCount|1.0 is 5.417046530978679
Validation MSE with key = unigram|preserve|wordCount|10.0 is 5.3487883610962035
Validation MSE with key = unigram|preserve|wordCount|100.0 is 5.309129178745707
Test MSE at unigram, preserve, wordCount and lambda = 100.0 is 5.206755660610267
Validation MSE with key = unigram|remove|tf_idf|0.01 is 5.333881336

Unigram model with preserve punctuations and TF IDF score performs better than all models on the test set. The above tables have the values for validation set MSE and test set MSE for a total of 48 configurations (40 for validation set and 8 for test set)